# Import libraries :

Disini kita akan mengimport Libraries yang akan di pakai pada dataset ini secara satu - persatu, Libraries yang dipakai adalah :

pandas yang di gunakan untuk membaca dataset yang berisi Label dan Deskripsi.

spacy yang di gunakan untuk melatih text classifier.

random yang di gunakan untuk menshuffle data dan membuat angka random.

pickle yang di gunakan untuk membuat serialisasi dan deserialisasi objek Python.

In [7]:
import pandas as pd
import spacy
import random
import pickle

# Load the Dataset :

setelah itu kita akan memanggil dataset melalui pandas dan memberikannya label yaitu, flower. lalu kita akan membuat nya memenunjukkan 90 baris pertama dari dataset yang telah di panggil

In [8]:
# Loading the flower data
flower = pd.read_csv('D:\\New folder\\Coding KB\\gatau\\DatasetBunga.csv')
flower.head(90)

,Label,Description,Text
0,Daisy,Bellis perennis adalah spesies Eropa yang umum...,bunga warna putih
1,Daisy,Bellis perennis adalah spesies Eropa yang umum...,bunga memiliki kelopak yang kecil
2,Daisy,Bellis perennis adalah spesies Eropa yang umum...,bunga putih dan kuning
3,Daisy,Bellis perennis adalah spesies Eropa yang umum...,bunga melingkari pusat kuning
4,Daisy,Bellis perennis adalah spesies Eropa yang umum...,bunga ada lingkaran kuning
...,...,...,...
77,Sunflower,Bunga matahari (Helianthus annuus L.) adalah t...,bunga yang kelopaknya warna kuning
78,Sunflower,Bunga matahari (Helianthus annuus L.) adalah t...,bunga yang ada di teletabis
79,Sunflower,Bunga matahari (Helianthus annuus L.) adalah t...,bunga yang melambangkan cinta sejati
80,Sunflower,Bunga matahari (Helianthus annuus L.) adalah t...,bunga yang ukuran nya besar


lalu kita akan membuat model NLP kosong menggunakan library spacy dan menambahkan Text Categorizer ke dalam model tersebut

In [11]:
# Create an empty model
nlp = spacy.blank("en")

# Add the TextCategorizer to the empty model
textcat = nlp.add_pipe("textcat")

# Add labels to the text classifier :

disini kita akan menambahkan label kepada Text Classifiernya, label nya sendiri berupa nama - nama bunga yang terdapat dalam Dataset.

In [14]:
# Add labels to text classifier
textcat.add_label("Daisy")
textcat.add_label("Tulips")
textcat.add_label("Dandelion")
textcat.add_label("Roses")
textcat.add_label("Sunflower")

0

# Prepare the training data :

disini kita akan melatih data yang telah kita dapat kan dari Dataset dengan cara memanggil kolom 'Text' dari Dataset lalu Ekstrak nilai teks mentah dari kolom tersebut menggunakan atribut ".values" dan menyimpan nya didalam variable train_text.

lalu kita akan meliterasi tiap label yang ada di 'label' yang terletak pada kolom DataFrame 'flower' menggunakan list comprehension. setelah itu kita akan membuat dictionary dengan 'cats' sebagai kata kunci nya.

In [17]:
train_texts = flower['Text'].values
train_labels = [{'cats': {'Daisy': label == 'Daisy',
                          'Tulips': label == 'Tulips',
                          'Dandelion': label == 'Dandelion',
                          'Roses': label == 'Roses',
                          'Sunflower': label == 'Sunflower'}} 
                for label in flower['Label']]

# Train the text classifier :

lalu kita akan menggabungkan 2 daftar yaitu train_texts dan train_labels, menjadi satu daftar tupel, di mana setiap tupel berisi deskripsi teks dan label terkait. Ini membuat data pelatihan terstruktur untuk pengklasifikasi teks.

setelah itu kita akan mencetak 90 elemen pertama dari train_datadaftar untuk memberikan gambaran sekilas tentang struktur dan isinya.

In [18]:
train_data = list(zip(train_texts, train_labels))
train_data[:90]

[('bunga warna putih',
  {'cats': {'Daisy': True,
    'Tulips': False,
    'Dandelion': False,
    'Roses': False,
    'Sunflower': False}}),
 ('bunga memiliki kelopak yang kecil',
  {'cats': {'Daisy': True,
    'Tulips': False,
    'Dandelion': False,
    'Roses': False,
    'Sunflower': False}}),
 ('bunga putih dan kuning',
  {'cats': {'Daisy': True,
    'Tulips': False,
    'Dandelion': False,
    'Roses': False,
    'Sunflower': False}}),
 ('bunga melingkari pusat kuning',
  {'cats': {'Daisy': True,
    'Tulips': False,
    'Dandelion': False,
    'Roses': False,
    'Sunflower': False}}),
 ('bunga ada lingkaran kuning',
  {'cats': {'Daisy': True,
    'Tulips': False,
    'Dandelion': False,
    'Roses': False,
    'Sunflower': False}}),
 ('bunga bisa Tumbuh di Mana Saja',
  {'cats': {'Daisy': True,
    'Tulips': False,
    'Dandelion': False,
    'Roses': False,
    'Sunflower': False}}),
 ('bunga memiki banyak Kegunaan',
  {'cats': {'Daisy': True,
    'Tulips': False,
    'Dandel

lalu kita akan mengimpor function dari spacy yaitu 

minibatch : Untuk membuat kumpulan data pelatihan, memungkinkan pembaruan model yang efisien.

Example : Untuk membuat contoh pelatihan dengan teks dan label untuk dipelajari oleh model.

setelah itu kita akan menetapkan benih acak menggunakan 

spacy.util.fix_random_seed(1): Memastikan reproduktifitas hasil dengan memperbaiki seed acak untuk pelatihan model.

baru kita memulai proses pelatihan model klasifikasi teks dan menyiapkan pengoptimal untuk memperbarui bobot model. setelah itu membuat minibatch yang berguna untuk membagi data pelatihan menjadi batch yang lebih kecil yang masing-masing terdiri dari 8 sampel untuk pemrosesan yang lebih efisien dan Iterasi melalu minibatch tersebut.

lalu kita proses setiap pasangan label teks dalam satu batch, lalu melakukan iterasi melalui setiap teks dan label terkait dalam satu batch.

In [19]:
from spacy.util import minibatch
from spacy.training.example import Example

spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

# Create the batch generator with batch size = 8
batches = minibatch(train_data, size=8)
# Iterate through minibatches
for batch in batches:
    # Each batch is a list of (text, label) 
    for text, labels in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, labels)
        nlp.update([example], sgd=optimizer)

# Predict the flower species :

nah disinilah bagian dimana kita mulai mempredict, tapi pertama kita menetapkan benih acak untuk random modul dan spaCy untuk memastikan reproduktifitas hasil dan perilaku pelatihan model yang konsisten. Kemudian, kita menginisialisasi pengoptimal untuk mengelola pembaruan model selama pelatihan. Selanjutnya, kita melakukan iterasi melalui proses pelatihan selama 10 epoch, mengacak data pelatihan secara acak dalam setiap periode untuk mencegah bias model terhadap pola data tertentu.

baru lah kita membagi data pelatihan yang diacak menjadi kelompok-kelompok kecil yang masing-masing terdiri dari 8 contoh. Hal ini membantu pembaruan model dan pemanfaatan sumber daya secara efisien dan mengulang setiap kumpulan contoh pelatihan.


dan yang terakhir melakukan Iterasi melalui setiap teks dan label terkait dalam satu batch untuk memperbarui model, dan mencetak kerugian setelah setiap periode.

In [20]:
random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            nlp.update([example], sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 12.740508515387774}
{'textcat': 19.15230168088908}
{'textcat': 22.513350671390185}
{'textcat': 23.19836127483824}
{'textcat': 23.284706786526318}
{'textcat': 23.28766885875311}
{'textcat': 23.287721248407617}
{'textcat': 23.287744482550146}
{'textcat': 23.28775769647239}
{'textcat': 23.287766276671448}


lalu untuk memprediksi jenis bunga dari deskripsi, kita pertama-tama membuat objek dokumen spaCy untuk setiap deskripsi. Kemudian, kita menggunakan model bahasa spaCy untuk memprediksi skor untuk setiap label untuk setiap dokumen. Skor tertinggi menunjukkan label yang paling mungkin untuk deskripsi.

In [21]:
texts = ["bunga warna merah"]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

print(scores)

[[4.0649160e-04 1.4522441e-03 1.7227696e-03 9.8311806e-01 1.3300375e-02]]


# Add descriptions for each label

untuk bagian ini penjelasannya sama seperti di cell sebelum nya kita hanya menambahkan deskripsi untuk tiap label, jadi jika saat "Prediction" yang berisi label di print dia juga akan mengeluarkan "Description" yaitu berupa penjelasan dari masing - masing bunga yang ada di Dataset.

In [ ]:
texts = ["bunga warna kuning"]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

 
    
# Add a dictionary to store descriptions for each label
label_descriptions = {
    "Daisy": "Bellis perennis adalah spesies Eropa yang umum dari daisy, dari familia Asteraceae, sering dianggap sebagai spesies archetypal dari nama itu. Bellis perennis asli dari Eropa barat, tengah dan utara, tetapi secara luas dinaturalisasi di sebagian besar daerah yang beriklim sedang termasuk Amerika dan Australasia. Secara umum, bunga daisy telah lama dikaitkan dengan simbol ketulusan, kemurnian, kelahiran kembali, dan keceriaan.",
    "Tulips": "Tulip (bahasa Latin: Tulipa) merupakan nama genus untuk 100 spesies tumbuhan berbunga yang termasuk ke dalam keluarga Liliaceae. Tulip berasal dari Asia Tengah, tumbuh liar di kawasan pegunungan Pamir dan pegunungan Hindu Kush dan stepa di Kazakhstan. Negeri Belanda terkenal sebagai negeri bunga tulip. Tulip juga merupakan bunga nasional Iran dan Turki..",
    "Dandelion": "Dandelion (Taraxacum) adalah genus besar dalam keluarga Asteraceae. Nama Randa Tapak sendiri biasanya merujuk kepada sebuah tumbuhan yang memiliki 'bunga-bunga' kecil yang dapat terbang ketika tertiup angin. Asal asli dari tumbuhan ini adalah Eropa dan Asia, tetapi sudah menyebar ke segala tempat. Yang disebut sebagai 'bunga' dari tumbuhan ini dapat menjadi semacam jam hayati yang secara teratur melepaskan banyak bijinya. Biji-biji ini sesungguhnya adalah buahnya.",
    "Roses": "Roses / rose adalah tumbuhan perdu, pohonnya berduri, bunganya berbau wangi dan berwarna indah, terdiri atas daun bunga yang bersusun:meliputi jenis, tumbuh tegak atau memanjat, batangnya berduri, bunganya beraneka warna, seperti merah, putih, merah jambu, merah tua, dan berbau harum. Mawar liar terdiri dari 100 spesies lebih, kebanyakan tumbuh di belahan bumi utara yang berudara sejuk. Spesies ini umumnya merupakan tanaman semak yang berduri atau tanaman memanjat yang tingginya bisa mencapai 2 sampai 5 meter. Walaupun jarang ditemui, tinggi tanaman mawar yang merambat di tanaman lain bisa mencapai 20 meter.",
    "Sunflower": "Bunga matahari (Helianthus annuus L.) adalah tumbuhan semusim dari suku kenikir-kenikiran (Asteraceae) yang populer, baik sebagai tanaman hias maupun tanaman penghasil minyak. Bunga tumbuhan ini sangat khas: besar, biasanya berwarna kuning terang, dengan kepala bunga yang besar (diameter bisa mencapai 30 cm). Bunga ini sebetulnya adalah bunga majemuk, tersusun dari ratusan hingga ribuan bunga kecil pada satu bongkol. Bunga Matahari juga memiliki perilaku khas, yaitu bunganya selalu menghadap / condong ke arah matahari atau heliotropisme. Orang Prancis menyebutnya tournesol atau 'Pengelana Matahari'. Namun, sifat ini disingkirkan pada berbagai kultivar baru untuk produksi minyak karena memakan banyak energi dan mengurangi hasil.",
}

# From the scores, find the label with the highest score/probability
predicted_labels = scores.argmax(axis=1)

# Print the predicted labels and their descriptions
for i, label in enumerate(predicted_labels):
    print(f"Prediction : {textcat.labels[label]}")
    print(f"Description: {label_descriptions.get(textcat.labels[label], 'Description not available')}")

# Save the text classifier

kita sudah masuk di bagian akhir yaitu bagian untuk save modelnya pertama kita akan mengimpor Objek TextCategorizer dari modul spacy.pipeline untuk digunakan dalam memprediksi kategori teks.

In [22]:
from spacy.pipeline import TextCategorizer
prediction = TextCategorizer

lalu kita menyimpan model pembelajaran mesin yang telah dilatih ke file flower.pkl menggunakan modul pickle.

In [23]:
import pickle
pickle_out = open("flower.pkl", "wb")
pickle.dump(prediction, pickle_out)
pickle_out.close()